In [31]:
import numpy as np
import soundfile as sf
import sounddevice as sd
from scipy.signal import lfilter
from scipy.io.wavfile import write

In [36]:
def escuchar_audio(nombre):
    # Lectura del archivo
    archivo, fs = sf.read(nombre)
    # Reproduce el sonido
    sd.play(archivo, samplerate=fs)
    sd.wait()

In [24]:
def custom_lfilter(b, a, x):
    # Longitudes de los coeficientes
    N = len(a)
    M = len(b)
    
    # Asegurarse de que a[0] es 1, si no, normalizar
    if a[0] != 1:
        a = [coef/a[0] for coef in a]
        b = [coef/a[0] for coef in b]
    
    # Inicializar la salida
    y = np.zeros_like(x)
    
    # Iterar sobre cada muestra de la señal
    for n in range(len(x)):
        y[n] = sum([b[k] * x[n-k] for k in range(M) if n-k >= 0])
        y[n] -= sum([a[k] * y[n-k] for k in range(1, N) if n-k >= 0])
    
    return y


In [65]:
# Lectura del archivo
archivo, fs = sf.read('./Sonidos/xylophon.mp3')

# Convertir el audio estéreo a mono si es necesario
if len(archivo.shape) > 1 and archivo.shape[1] == 2:
    archivo = np.mean(archivo, axis=1)

# Parámetros del filtro
alpha = 0.6
k_sinfs = 0.05

# Ajuste para el retardo k
k = int(fs * k_sinfs)

# Se crea un vector de k elementos lleno de ceros para implementar el retardo hasta k muestras
polos = np.zeros(k)

# Se coloca el y(n)
polos[0] = 1

# Se coloca el retardo a la muestra y(n-k)
polos[k-1] = alpha

# Se coloca la atenuación (1 - alpha) a la muestra x(n)
ceros = [1 - alpha]


print(polos.shape)
print(ceros)

(2205,)
[0.4]


In [49]:
# Aplica el filtro
salida_filtro = custom_lfilter(ceros, polos, archivo)

# Reproduce el sonido
sd.play(salida_filtro, samplerate=fs)
sd.wait()
# Guarda el archivo
write("dog_conFiltro.wav", fs, salida_filtro)

# Eliminación de reverberación
input('Presione enter para escuchar reverberación eliminada')

# Se invierten Y y X al ser un proceso inverso
salida_filtro = lfilter(polos, ceros, salida_filtro)

# Reproduce el sonido
sd.play(salida_filtro, samplerate=fs)
sd.wait()

Presione enter para escuchar reverberación eliminada 


In [61]:
escuchar_audio("./Sonidos/xylophon_conFiltro.wav")